<a href="https://colab.research.google.com/github/tikik/First-Git/blob/master/Homework_6825_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# # Patient-Level Train/Validation Split on CheXpert Dataset

## Objectives

Perform a **patient-level 80/20 train/validation split** on the CheXpert dataset from Hugging Face.

- Ensure **no data leakage**: images from the same patient only appear in one split.
- Use only **frontal view** images.

---

## Data Source

- Dataset: [danjacobellis/chexpert](https://huggingface.co/datasets/danjacobellis/chexpert) from Hugging Face.
- Format: Parquet (`.parquet`).
- Environment: Google Colab (due to memory/time constraints).
- Streaming mode is used to avoid memory and rate limit issues.
---

## Preprocessing Steps

### 1. Load Dataset with Retry Logic

kljh,b from datasets import load_dataset
import time

for attempt in range(5):
    try:
        dataset = load_dataset("danjacobellis/chexpert", split="train")
        break  # Success
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        time.sleep(2 ** attempt)





In [36]:
import pandas as pd
import dask.dataframe as dd

splits = {'train': 'data/train-*.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
df = dd.read_parquet("hf://datasets/danjacobellis/chexpert/" + splits["train"])


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import dask.dataframe as dd

# Load the training data from HF parquet files
df = dd.read_parquet("hf://datasets/danjacobellis/chexpert/data/train-*.parquet")

In [22]:
# Extract patient ID from 'Path' column, handling potential errors
df['PatientID'] = df['Path'].str.split('/').str[2]

In [10]:

from datasets import load_dataset
import time

for attempt in range(5):
    try:
        dataset = load_dataset("danjacobellis/chexpert", split="train")
        break  # Success, exit loop
    except Exception as e:
        print(f"Attempt {attempt + 1} failed: {e}")
        wait_time = 2 ** attempt
        print(f"Retrying in {wait_time} seconds...")
        time.sleep(wait_time)


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

data/train-00000-of-00023.parquet:   0%|          | 0.00/487M [00:00<?, ?B/s]

data/train-00001-of-00023.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

...

data/train-00022-of-00023.parquet:   0%|          | 0.00/516M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/223414 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/234 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [13]:
df = pd.DataFrame.from_records(df.head(50000))

/usr/local/lib/python3.12/dist-packages/dask/dataframe/core.py:382: UserWarning: Insufficient elements for `head`. 50000 elements requested, only 9714 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(
/tmp/ipython-input-2302168727.py:1: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(df.head(50000))


In [15]:
# Filter for frontal views (assuming 'Frontal/Lateral' column contains 1 for frontal)
df = df[df['Frontal/Lateral'] == 1]

In [24]:
unique_patients = df['PatientID'].unique()
train_patients, val_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

In [31]:
display(df['Path'].head())



,Path
3,CheXpert-v1.0-small/train/patient00002/study1/...
6,CheXpert-v1.0-small/train/patient00004/study1/...
8,CheXpert-v1.0-small/train/patient00005/study1/...
17,CheXpert-v1.0-small/train/patient00009/study1/...
19,CheXpert-v1.0-small/train/patient00010/study1/...


In [25]:
train_patients, val_patients = train_test_split(unique_patients, test_size=0.2, random_state=42)

# New Section

In [26]:
df['split'] = df['PatientID'].apply(lambda x: 'train' if x in train_patients else 'val')

In [27]:
train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']

In [33]:
print(len(train_df), len(val_df))


1601 386


In [32]:
train_df.to_parquet("chexpert_train_frontal.parquet")
val_df.to_parquet("chexpert_val_frontal.parquet")
import os
print(os.getcwd())
print(os.listdir())

/content
['.config', 'chexpert_train_frontal.parquet', 'chexpert_val_frontal.parquet', 'sample_data']


In [28]:
print("Train size:", len(train_df))
print("Validation size:", len(val_df))

Train size: 1601
Validation size: 386
